### JOIN

- When the data you want is spread across multiple tables, Then, We use __JOIN__
- ie to extract data from 2 tables having any one same column

### WHERE

- a WHERE clause can limit your results to rows with certain text using the __LIKE__ feature

- query_1 = """
          SELECT *
          FROM `  `
          WHERE Name LIKE 'Ripley'     """
          
- We can use __%__ as __Wildcard__ for any number of characters

- query_2 = """
            SELECT *
            FROM `    ` 
            WHERE Name LIKE '%ipl%'

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'smiling-audio-287409-77f38c77b232.json'
from google.cloud import bigquery

### Q1: How many files are covered by each type of software license?

GitHub is the most popular place to collaborate on software projects. A GitHub repository (or repo) is a collection of files associated with a specific project.

Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on how they are used. For our example, we're going to look at how many different files have been released under each license.

We'll work with two tables in the database. The first table is the licenses table, which provides the name of each GitHub repo (in the repo_name column) and its corresponding license. Here's a view of the first five rows.

In [2]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results=5).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:17: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.


,repo_name,license
0,lostlovespell/mamahealer,artistic-2.0
1,gitpan/Protocol-Notifo,artistic-2.0
2,Mike9013/RogueLite,artistic-2.0
3,rmp/libclearpress-perl,artistic-2.0
4,MARTIMM/pod-render,artistic-2.0


In [4]:
# Second Table is Sample_files

files_ref = dataset_ref.table("sample_files")
files_table = client.get_table(files_ref)
client.list_rows(files_table, max_results=5).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """


,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll


In [8]:
# Now, we will write a query to determine how many files are related to each other

query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """

safe_config = bigquery.QueryJobConfig(maximum_bytes_billed = 10**10)
query_job = client.query(query, job_config=safe_config)
files_count_by_license = query_job.to_dataframe()

files_count_by_license.head()

,license,number_of_files
0,mit,20432437
1,gpl-2.0,16866825
2,apache-2.0,7114020
3,gpl-3.0,4936531
4,bsd-3-clause,2926635


## Q2

In [9]:
# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

### 1) Explore the data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

*Hint*: Tab completion is helpful whenever you can't remember a command. Type `client.` and then hit the tab key. Don't forget the period before hitting tab.

In [10]:
tables = list(client.list_tables(dataset))

# Get a list of available tables 
list_of_tables = [table.table_id for table in tables]


print(list_of_tables)

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


### 2) Review relevant tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look.

In [11]:
answers_table_ref = dataset_ref.table("posts_answers")

answers_table = client.get_table(answers_table_ref)

client.list_rows(answers_table, max_results=5).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,61514077,None,"<p>You can use ""break"" to break the loop, whic...",None,None,0,None,2020-04-30 00:45:21.430000+00:00,None,2020-04-30 00:45:21.430000+00:00,None,None,None,user13366406,None,7951690,2,0,None,None
1,61514948,None,"<p>Okay I solved it, I had the incorrect Strin...",None,None,0,None,2020-04-30 02:31:51.440000+00:00,None,2020-04-30 02:31:51.440000+00:00,None,None,None,user9969622,None,61470541,2,0,None,None
2,61518060,None,<p>there's no such library available that sati...,None,None,0,None,2020-04-30 07:29:01.820000+00:00,None,2020-04-30 07:29:01.820000+00:00,None,None,None,user13317089,None,61311006,2,0,None,None
3,61520287,None,"<p>{\n ""error"": {\n ""code"": ""Forbidd...",None,None,1,None,2020-04-30 09:34:29.533000+00:00,None,2020-04-30 09:34:29.533000+00:00,None,None,None,user4746302,None,61519229,2,0,None,None
4,61521694,None,"<p>Solveded like now its opening, because I re...",None,None,0,None,2020-04-30 10:50:25.500000+00:00,None,2020-04-30 10:50:25.500000+00:00,None,None,None,user13388525,None,61520555,2,0,None,None


It isn't clear yet how to find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Looking at `posts_questions` table. 

In [12]:
questions_table_ref = dataset_ref.table("posts_questions")

questions_table = client.get_table(questions_table_ref)

client.list_rows(questions_table, max_results=5).to_dataframe()

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:5: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,62111964,Sending custom params from client on silent re...,<p>Is there any way we could pass extra parame...,None,0,0,None,2020-05-31 05:30:55.990000+00:00,None,2020-05-31 05:30:55.990000+00:00,None,None,None,None,10062258,None,1,0,oauth-2.0|identityserver4|openiddict|angular-o...,1
1,61815842,What Cloud Storage or Service to use for excha...,<p>Storage has to be accessed by C++ and NodeJ...,None,0,0,None,2020-05-15 09:17:23.447000+00:00,None,2020-05-15 09:17:23.447000+00:00,None,None,None,None,2182080,None,1,0,cloud-storage|real-time-data,2
2,61833254,How to capture redirected link and send to std...,<p>Please I need help. Assuming I have this li...,None,0,0,None,2020-05-16 07:19:58.070000+00:00,None,2020-05-16 07:19:58.070000+00:00,None,None,None,None,11375445,None,1,0,shell|url|curl|redirect,2
3,61935894,"where i should add ""clickAnalytics: true"" on f...",<p>I want to track/add data for conversion rat...,None,0,0,None,2020-05-21 13:42:25.997000+00:00,None,2020-05-21 13:42:25.997000+00:00,None,None,None,None,13120736,None,1,0,analytics|magento-1.9,2
4,61936080,How to avoid uploaded file name encrypt in nodejs,<p>In my code uploaded file name saved in loca...,None,0,0,None,2020-05-21 13:51:52.813000+00:00,None,2020-05-21 13:51:52.813000+00:00,None,None,None,None,13576535,None,1,0,node.js|angular8,2


`posts_questions` has a column called tags which lists the topics/technologies each question is about.

`posts_questions` has a column called __parent_id__ which identifies the ID of the question each answer is responding to. `posts_answers` also has an __owner_user_id__ column which specifies the ID of the user who answered the question.

You can join these two tables to:

- determine the tags for each answer, and then
- select the owner_user_id of the answers on the desired tag.

### 3) Selecting the right questions

A lot of this data is text. 

We'll explore one last technique in this course which you can apply to this text.

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature.

### Write a query that selects the `id`, `title` and `owner_user_id` columns from the `posts_questions` table. 
- Restrict the results to rows that contain the word "bigquery" in the `tags` column. 
- Include rows where there is other text in addition to the word "bigquery" (e.g., if a row has a tag "bigquery-sql", your results should include that too).

In [13]:
questions_query = """
                  SELECT id, title, owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_questions`
                  WHERE tags LIKE '%bigquery%'
                  """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
questions_query_job = client.query(questions_query, job_config= safe_config)

# API request - run the query, and return a pandas DataFrame
questions_results = questions_query_job.to_dataframe() 

# Preview results
print(questions_results.head())


         id                                              title  owner_user_id
0  17997089      BigQuery rank() function - Resources exceeded      2276668.0
1  18116803  Getting 401 uploading file into a table with a...       508960.0
2  18050595  Workaround needed for the incomplete implement...      2264178.0
3  17884358      Insufficient data written when inserting rows      1752620.0
4  18091168                 Can't upload to BigQuery from Perl       585411.0


### 4)  join
Now that you have a query to select questions on any given topic (in this case, you chose "bigquery"), you can find the answers to those questions with a **JOIN**.  

Write a query that returns the `id`, `body` and `owner_user_id` columns from the `posts_answers` table for answers to "bigquery"-related questions. 
- You should have one row in your results for each answer to a question that has "bigquery" in the tags.  
- Remember you can get the tags for a question from the `tags` column in the `posts_questions` table.

Here's a reminder of what a **JOIN** looked like in the tutorial:
```
query = """
        SELECT p.Name AS Pet_Name, o.Name AS Owner_Name
        FROM `bigquery-public-data.pet_records.pets` as p
        INNER JOIN `bigquery-public-data.pet_records.owners` as o 
            ON p.ID = o.Pet_ID
        """
```

It may be useful to scroll up and review the first several rows of the `posts_answers` and `posts_questions` tables.  

In [14]:
answers_query = """
                SELECT a.id, a.body, a.owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                    ON q.id = a.parent_id
                WHERE q.tags LIKE '%bigquery%'
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**12)
answers_query_job = client.query(answers_query, job_config= safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
answers_results = answers_query_job.to_dataframe() # Your code goes here

# Preview results
print(answers_results.head())

         id                                               body  owner_user_id
0  29358676  <p>This tutorial: <a href="http://examples.jav...      4271541.0
1  29495431  <p>The return value of the BigQuery.Jobs.inser...      3906190.0
2  29650843  <p>I think you should add the Kind field [1]. ...      2065479.0
3  29718738  <p>This is a common need when building a datab...      2221001.0
4  29855525  <p>Make sure you are providing appropriate val...      4743732.0


### 5) Answer the question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that has a single row for each user who answered at least one question with a tag that includes the string "bigquery". Your results should have two columns:
- `user_id` - contains the `owner_user_id` column from the `posts_answers` table
- `number_of_answers` - contains the number of answers the user has written to "bigquery"-related questions

In [15]:
bigquery_experts_query = """
                        SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                        FROM `bigquery-public-data.stackoverflow.posts_questions` AS q 
                        INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                                    ON q.id = a.parent_id
                        WHERE q.tags LIKE '%bigquery%' 
                        GROUP BY a.owner_user_id
                        
                            
"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
bigquery_experts_query_job = client.query(bigquery_experts_query, job_config=safe_config) # Your code goes here

# API request - run the query, and return a pandas DataFrame
bigquery_experts_results = bigquery_experts_query_job.to_dataframe() # Your code goes here

# Preview results
print(bigquery_experts_results.head())

      user_id  number_of_answers
0    328036.0                  8
1   4605070.0                  1
2  11453690.0                  1
3  11765109.0                  1
4   5573294.0                  1


### 6) Building a more generally useful service

How could you convert what you've done to a general function a website could call on the backend to get experts on any topic?  


In [16]:
def expert_finder(topic, client):
    '''
    Returns a DataFrame with the user IDs who have written Stack Overflow answers on a topic.

    Inputs:
        topic: A string with the topic of interest
        client: A Client object that specifies the connection to the Stack Overflow dataset

    Outputs:
        results: A DataFrame with columns for user_id and number_of_answers. Follows similar logic to bigquery_experts_results shown above.
    '''
    my_query = """
               SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
               FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
               INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                   ON q.id = a.parent_Id
               WHERE q.tags like '%{topic}%'
               GROUP BY a.owner_user_id
               """

    # Set up the query (a real service would have good error handling for 
    # queries that scan too much data)
    safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)      
    my_query_job = client.query(my_query, job_config=safe_config)

    # API request - run the query, and return a pandas DataFrame
    results = my_query_job.to_dataframe()

    return results